In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [ ]:
import json
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout

In [ ]:
# Téléchargement des ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Chargement des données depuis le fichier JSON
with open('Intents.json', 'r') as file:
    data = json.load(file)

In [ ]:
# Initialisation du lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Chargement des stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
# Tokenisation, suppression des stop words et lemmatisation
tokenized_patterns = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        words = word_tokenize(pattern.lower())
        words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
        tokenized_patterns.append((words, intent['tag']))

In [ ]:
# Prétraitement des données
all_words = []
tags = []
xy = []
for (pattern_words, tag) in tokenized_patterns:
    all_words.extend(pattern_words)
    xy.append((pattern_words, tag))
    if tag not in tags:
        tags.append(tag)

In [ ]:
# Création du vocabulaire
vocab_size = 1000
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(all_words)

In [ ]:
# Conversion en séquences d'indices
X_train = []
y_train = []
for (pattern_words, tag) in xy:
    sequence = tokenizer.texts_to_sequences([pattern_words])[0]
    X_train.append(sequence)
    y_train.append(tags.index(tag))

X_train = pad_sequences(X_train, padding='post')
y_train = np.array(y_train)

In [ ]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Définition des paramètres du modèle
embedding_dim = 16
max_length = len(X_train[0])
output_dim = len(tags)

In [ ]:
# Modèle LSTM
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(128),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(output_dim, activation='softmax')
])

In [ ]:
# Initialisation du modèle
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Entraînement du modèle
model.fit(X_train, y_train, epochs=500, batch_size=8, verbose=1)

Epoch 1/500
50/50 [==============================] - 8s 45ms/step - loss: 4.6097 - accuracy: 0.0050
Epoch 2/500
50/50 [==============================] - 1s 11ms/step - loss: 4.6053 - accuracy: 0.0075
Epoch 3/500
50/50 [==============================] - 1s 11ms/step - loss: 4.6016 - accuracy: 0.0150
Epoch 4/500
50/50 [==============================] - 0s 8ms/step - loss: 4.6003 - accuracy: 0.0175
Epoch 5/500
50/50 [==============================] - 0s 5ms/step - loss: 4.5995 - accuracy: 0.0075
Epoch 6/500
50/50 [==============================] - 1s 11ms/step - loss: 4.5293 - accuracy: 0.0200
Epoch 7/500
50/50 [==============================] - 0s 8ms/step - loss: 4.4097 - accuracy: 0.0125
Epoch 8/500
50/50 [==============================] - 0s 5ms/step - loss: 4.2988 - accuracy: 0.0250
Epoch 9/500
50/50 [==============================] - 0s 5ms/step - loss: 4.1609 - accuracy: 0.0150
Epoch 10/500
50/50 [==============================] - 0s 5ms/step - loss: 4.1229 - accuracy: 0.0225
Epoch

In [ ]:
# Évaluation du modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy:", accuracy)

4/4 [==============================] - 1s 7ms/step - loss: 29.0181 - accuracy: 0.0800
Test Accuracy: 0.07999999821186066


In [ ]:
# Conversation avec le chatbot
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        sentence = input("You: ")
        if sentence.lower() == "quit":
            break

        sequence = tokenizer.texts_to_sequences([word_tokenize(sentence.lower())])[0]
        sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
        prediction = model.predict(sequence)
        predicted_tag = tags[np.argmax(prediction)]

        for intent in data['intents']:
            if intent['tag'] == predicted_tag:
                print("Bot:", np.random.choice(intent['responses']))
                break

In [ ]:
chat()

Start talking with the bot (type quit to stop)!
You: comment appliquer la pédagogie avec les enfants
1/1 [==============================] - 0s 396ms/step
Bot: Utilisez des logiciels comme NVivo, ATLAS.ti ou MAXQDA pour faciliter l'analyse de contenu.
You: quelle est la relation entre la sociologie et l'éducation
1/1 [==============================] - 0s 26ms/step
Bot: La sociologie de l'éducation examine les façons dont l'éducation affecte et est affectée par la société et les interactions sociales.
